In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
%matplotlib inline

In [3]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# Used by other groups: pd.read_csv(data_path, sep="\t",encoding='utf-8',  escapechar='\\', quoting=csv.QUOTE_NONE, header=None, na_values='N')

In [4]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

In [5]:
tw.head()

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,17341045.0,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586.0,508.0,9016.0,"Bern, Switzerland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9514846412,7198282.0,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820.0,703.0,4687.0,"Zurich, Switzerland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9516574359,14657884.0,2010-02-23 07:34:25,It has been a week or so.. and today I just co...,6.13396,46.1951,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def extract_hashtags(text):
    if '#' not in text:
        return None
    return [tag.strip("#") for tag in text.split() if tag.startswith("#")]

In [7]:
tw['hashtag'] = np.nan
tw.head()
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw[pd.notnull(tw['hashtag'])]
len(twh)

79537

In [8]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 79537 tweets to 79537 tweets.


In [9]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

12:57:47 Removing bad dates...
12:57:47 Finished.


In [10]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

12:57:47 Starting to examine dates...
12:57:47 There are 0 dates that cannot be transformed.


In [37]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

13:08:07 Starting copy...
13:08:07 Converting to datetime...
13:08:07 Setting up new indices...
13:08:07 Deleting old "createdAt" column...
13:08:07 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,14657884.0,"Magic spells run off after midnight, I guess s...",6.1387,46.175,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[fb]
2010-02-23 11:28:27,9521789689,9962022.0,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[yam]


In [38]:
twt = tw5.head()
twt.hashtag

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11         [24, VfB.]
2010-02-23 18:19:03    [iPhoneography]
2010-02-23 18:31:46     [partnermonth]
Name: hashtag, dtype: object

In [39]:
tw5.hashtag = tw5.hashtag.apply(lambda x: ', '.join(x).lower())
tw5.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11         24, vfb.
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

In [40]:
tw5.dropna(subset=['longitude'], inplace=True)
tw5.dropna(subset=['latitude'], inplace=True)

In [41]:
tw5.latitude = tw5.latitude.apply(float)
tw5.longitude = tw5.longitude.apply(float)

In [52]:
gp.get_group('hollande')['id'].resample('D').count()

createdAt
2012-01-23    1
2012-01-24    0
2012-01-25    0
2012-01-26    0
2012-01-27    0
2012-01-28    0
2012-01-29    0
2012-01-30    0
2012-01-31    0
2012-02-01    0
2012-02-02    0
2012-02-03    0
2012-02-04    0
2012-02-05    0
2012-02-06    0
2012-02-07    0
2012-02-08    0
2012-02-09    0
2012-02-10    0
2012-02-11    0
2012-02-12    0
2012-02-13    0
2012-02-14    0
2012-02-15    0
2012-02-16    0
2012-02-17    0
2012-02-18    0
2012-02-19    0
2012-02-20    0
2012-02-21    0
             ..
2012-04-20    1
2012-04-21    0
2012-04-22    1
2012-04-23    1
2012-04-24    0
2012-04-25    0
2012-04-26    1
2012-04-27    0
2012-04-28    0
2012-04-29    0
2012-04-30    0
2012-05-01    2
2012-05-02    7
2012-05-03    2
2012-05-04    2
2012-05-05    4
2012-05-06    8
2012-05-07    1
2012-05-08    2
2012-05-09    0
2012-05-10    0
2012-05-11    0
2012-05-12    0
2012-05-13    0
2012-05-14    1
2012-05-15    2
2012-05-16    0
2012-05-17    0
2012-05-18    0
2012-05-19    1
Freq: D, Name:

In [43]:
delimiter = '_$$$_'

In [44]:
 str_join = lambda x: delimiter.join(x)

In [45]:
del dictionary

In [46]:
def aggDate(df):
    test = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : str_join})
    test['numberOfTweets'] = test['hashtag'].apply(lambda x: len(x.split(delimiter)))
    return test

In [54]:
gp = tw5.groupby('hashtag')

dictionary = {hashtag : aggDate(df) for hashtag, df in gp}

In [48]:
len(dictionary)

29102

In [50]:
dictionary['hollande']

,text,latitude,hashtag,longitude,numberOfTweets
2012-01-23,"@ParisMatch comme il se doit ?? Pourquoi, Fran...",46.52130,hollande,6.633350,1
2012-04-20,"“@Marion_Z : ""Les françaises sont belles mais ...",46.52430,hollande,6.865800,1
2012-04-22,Mtn imagine Hollande il va au Pays-Bas. A l'aé...,46.68710,hollande,6.916410,1
2012-04-23,Qg de campagne #hollande le changement avec fa...,47.24330,hollande,5.997970,1
2012-04-26,@SARKOZY_2012 président franchement il est pl...,46.58680,hollande,6.602170,1
2012-05-01,"@nk_m traité sournoisement #Hollande de : ""Ren...",46.52420,hollande_$$$_hollande,6.866035,2
2012-05-02,#Hollande sait que s'il remonte la TVA sur la ...,46.58650,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.601790,7
2012-05-03,C'est @NicolasSarkozy qui gagnera car #Holland...,46.67720,hollande_$$$_hollande,7.765290,2
2012-05-04,"#Hollande est aimé des jeunes , du moins de ce...",47.62670,hollande_$$$_hollande,6.224710,2
2012-05-05,Dans ma ville l UMP ne communique que sur le d...,47.23960,hollande_$$$_hollande_$$$_hollande_$$$_hollande,6.035700,4


In [56]:
t = 0
for [h,df] in dictionary.items():
    t = t + 1
    if (t%10000 == 0):
        print(t)
    if len(df) > 3:
        threshold = df['numberOfTweets'].mean() + 2 * df['numberOfTweets'].std()
        df['event'] = df.numberOfTweets.apply(lambda x: x>threshold)

10000
20000


In [58]:
dictionary['hollande']

,text,latitude,hashtag,longitude,numberOfTweets,event
2012-01-23,"@ParisMatch comme il se doit ?? Pourquoi, Fran...",46.52130,hollande,6.633350,1,False
2012-04-20,"“@Marion_Z : ""Les françaises sont belles mais ...",46.52430,hollande,6.865800,1,False
2012-04-22,Mtn imagine Hollande il va au Pays-Bas. A l'aé...,46.68710,hollande,6.916410,1,False
2012-04-23,Qg de campagne #hollande le changement avec fa...,47.24330,hollande,5.997970,1,False
2012-04-26,@SARKOZY_2012 président franchement il est pl...,46.58680,hollande,6.602170,1,False
2012-05-01,"@nk_m traité sournoisement #Hollande de : ""Ren...",46.52420,hollande_$$$_hollande,6.866035,2,False
2012-05-02,#Hollande sait que s'il remonte la TVA sur la ...,46.58650,hollande_$$$_hollande_$$$_hollande_$$$_holland...,6.601790,7,True
2012-05-03,C'est @NicolasSarkozy qui gagnera car #Holland...,46.67720,hollande_$$$_hollande,7.765290,2,False
2012-05-04,"#Hollande est aimé des jeunes , du moins de ce...",47.62670,hollande_$$$_hollande,6.224710,2,False
2012-05-05,Dans ma ville l UMP ne communique que sur le d...,47.23960,hollande_$$$_hollande_$$$_hollande_$$$_hollande,6.035700,4,False
